In [7]:
# IMPORTING REQUIRED LIBRARIES
import torch


In [ ]:
#Create Hamiltonian For Zero Pi 

#Variables
ECS = 5
ECJ = 10
ncut = 30
ng = 5
dCJ = 2
dEJ = 2

#Common
def first_derivative_matrix(prefactor):
    #NEED TO FILL IN
    return

def second_derivative_matrix(prefactor):
    #NEED TO FILL IN
    return


pt_count = 5 #NOT SURE WHAT THIS IS? (to do with discretisation)
delta_x = 0.1 #NOT SURE WHAT THIS IS? (to do with discretisation)
dim_theta = 2 * ncut + 1


#Kinetic Part

def kinetic_matrix():

    identity_phi = torch.eye(pt_count)

    identity_theta = torch.eye(dim_theta)

    i_d_dphi_operator = torch.kron(first_derivative_matrix(prefactor=1j), identity_theta )

    kinetic_matrix_phi = second_derivative_matrix(prefactor =  -2.0*ECJ)

    ##
    diag_elements = 2.0 * ECS * torch.square(torch.arange(-ncut + ng, ncut + 1 + ng))
    kinetic_matrix_theta = torch.diag_embed(diag_elements, [0])  ### Need to figure out what is spare.dia_matrix is doing
    ##


    #
    diag_elements = torch.arange(-ncut, ncut + 1)
    n_theta_matrix= torch.diag_embed(diag_elements, [0])  ### Need to figure out what is spare.dia_matrix is doing
    #### native = sparse.kron(self._identity_phi(), n_theta_matrix, format="csc")
    ##### n_theta_operator = process_op(native_op=native, energy_esys=energy_esys)
    #

    kinetic_matrix = torch.kron(kinetic_matrix_phi, identity_theta)+ torch.kron(identity_phi, kinetic_matrix_theta)

    if dCJ != 0:
        kinetic_matrix -= (
                    2.0
                    * ECS
                    * dCJ
                    * i_d_dphi_operator()
                    * n_theta_operator()
        )
    return kinetic_matrix


#Potential Part 

def potential_matrix():

    grid_linspace = self.grid.make_linspace()
    phi_inductive_vals = self.EL * np.square(grid_linspace)
    phi_inductive_potential = sparse.dia_matrix(
            (phi_inductive_vals, [0]), shape=(pt_count, pt_count)
        ).tocsc()


    phi_cos_vals = np.cos(grid_linspace - 2.0 * np.pi * self.flux / 2.0)
    phi_cos_potential = sparse.dia_matrix(
            (phi_cos_vals, [0]), shape=(pt_count, pt_count)
        ).tocsc()

    phi_sin_vals = np.sin(grid_linspace - 2.0 * np.pi * self.flux / 2.0)
    phi_sin_potential = sparse.dia_matrix(
            (phi_sin_vals, [0]), shape=(pt_count, pt_count)
        ).tocsc()

    theta_cos_potential = (
            -self.EJ
            * (
                sparse.dia_matrix(
                    ([1.0] * dim_theta, [-1]), shape=(dim_theta, dim_theta)
                )
                + sparse.dia_matrix(
                    ([1.0] * dim_theta, [1]), shape=(dim_theta, dim_theta)
                )
            )
        ).tocsc()

    potential_mat = (
            sparse.kron(phi_cos_potential, theta_cos_potential, format="csc")
            + sparse.kron(phi_inductive_potential, self._identity_theta(), format="csc")
            + 2
            * self.EJ
            * sparse.kron(self._identity_phi(), self._identity_theta(), format="csc")
        )
        
        
    if dEJ != 0:
        potential_mat += (
                EJ
                * dEJ
                * sparse.kron(phi_sin_potential, self._identity_theta(), format="csc")
                * self.sin_theta_operator()
            )
    return potential_mat